# SysML → USD Digital Assembly Pipeline

This notebook demonstrates the full **S24 pipeline**:

1. Parse SysML v2 models (system + materials)
2. Convert to structured JSON representations
3. Vet and validate the system graph & material references
4. Generate a USD material library from SysML-defined properties
5. Generate USD geometry and components with material bindings
6. Assemble the full system hierarchy into a USD scene

### Architecture Overview

- **SysML** = Source of Truth (structure, hierarchy, materials identity + physical properties)
- **USD** = Integration layer (geometry + material look + physical data combined)
- **Omniverse** = Visualization & spatial exploration (later)

The output is a USD assembly suitable for visualization (usdview / Omniverse), simulation coupling, and digital twin workflows.

## Setup

In [ ]:
# Standard library
import os
import json
from pathlib import Path

# Verify USD is available
from pxr import Usd, UsdShade
print("USD loaded successfully")

# S24 pipeline imports
from S24.sysml.exporter import sysml_to_json, write_json, sysml_to_materials, write_materials_json
from S24.jsonio.vetting import VettingProc
from S24.usd.builder import USDBuilder
from S24.usd.material_library import generate_material_library

## Project Paths

We explicitly define all paths so the pipeline is reproducible and independent of the working directory.

In [ ]:
ROOT = Path.cwd().parent  # repo root (assuming notebook is in notebooks/)

# SysML sources
SYSML_DIR       = ROOT / "database" / "sysml"
SYSML_SYSTEM    = SYSML_DIR / "habitation.sysml"
SYSML_MATERIALS = SYSML_DIR / "materials.sysml"

# JSON outputs
JSON_DIR          = ROOT / "database" / "json"
JSON_PARTS        = JSON_DIR / "habmod.json"
JSON_MATERIALS    = JSON_DIR / "materials.json"

# USD outputs
ASSETS_DIR        = ROOT / "database" / "assets"
MAT_LIBRARY_PATH  = ASSETS_DIR / "mtl" / "lunar_materials.usda"
SCENES_DIR        = ROOT / "database" / "scenes"
SCENE_FILE        = SCENES_DIR / "HabitationAssembly.usda"

# Ensure directories exist
for d in [SYSML_DIR, JSON_DIR, ASSETS_DIR / "mtl", SCENES_DIR]:
    d.mkdir(parents=True, exist_ok=True)

print("SysML system: ", SYSML_SYSTEM)
print("SysML materials:", SYSML_MATERIALS)
print("JSON parts:     ", JSON_PARTS)
print("JSON materials: ", JSON_MATERIALS)
print("Material library:", MAT_LIBRARY_PATH)
print("Scene output:   ", SCENE_FILE)

## Step 1 — Load SysML Models

We load two SysML v2 models:
- **habitation.sysml**: system structure, hierarchy, dimensions, and `materialRef` identifiers
- **materials.sysml**: material definitions with physical properties (density, thermal conductivity, etc.)

SysML is the **source of truth** for all engineering data.

In [ ]:
# Load system model
sysml_system_text = SYSML_SYSTEM.read_text(encoding="utf-8")
print("=== System Model (first 600 chars) ===")
print(sysml_system_text[:600], "...")

In [ ]:
# Load materials model
sysml_materials_text = SYSML_MATERIALS.read_text(encoding="utf-8")
print("=== Materials Model (first 600 chars) ===")
print(sysml_materials_text[:600], "...")

## Step 2 — Export SysML to JSON

### 2a. System parts

The system model is converted into a flat JSON list of parts with:
- Explicit parent/children relationships
- Dimensions in SI units
- Evaluated attributes
- `materialRef` identifiers (stable IDs, not file paths)

In [ ]:
parts_json = sysml_to_json(sysml_system_text, namespace="lunarspaceport1")
write_json(parts_json, str(JSON_PARTS))

print(f"Exported {len(parts_json)} parts to {JSON_PARTS.name}")
print()

# Show first part
print(json.dumps(parts_json[0], indent=2))

### 2b. Material definitions

Material physical properties are exported from SysML into a dedicated `materials.json`.
These properties include density, Young's modulus, thermal conductivity, etc.

In [ ]:
materials = sysml_to_materials(sysml_materials_text)
write_materials_json(materials, str(JSON_MATERIALS))

print(f"Exported {len(materials)} materials to {JSON_MATERIALS.name}")
print()

for mat in materials:
    print(f"  - {mat['materialId']}")
    for k, v in mat.items():
        if k != 'materialId':
            print(f"      {k}: {v}")

## Step 3 — Vet JSON Representation

We perform structural and semantic validation:
- Required fields present
- Unique part names
- Parent–child consistency (bidirectional)
- Cycle detection
- Single connected system graph

In [ ]:
vetting = VettingProc(str(JSON_PARTS))
vetted_parts = vetting.by_name

print(f"Vetted {len(vetted_parts)} parts:")
for name, vp in vetted_parts.items():
    print(f"  - {name} (material: {vp.material_ref}, parent: {vp.parent})")

## Step 4 — Vet Material References

We verify that every `materialRef` used in a part actually exists in the material library.
This prevents broken bindings at the USD level.

In [ ]:
known_material_ids = {m["materialId"] for m in materials}

print(f"Known materials: {sorted(known_material_ids)}")
print()

all_valid = True
for name, vp in vetted_parts.items():
    if vp.material_ref not in known_material_ids:
        print(f"  ERROR: Part '{name}' references unknown material '{vp.material_ref}'")
        all_valid = False
    else:
        print(f"  OK: {name} -> {vp.material_ref}")

if all_valid:
    print("\nAll material references are valid.")
else:
    raise ValueError("Material vetting failed — see errors above.")

## Step 5 — Generate USD Material Library

The material library is **auto-generated** from SysML-defined properties.
Each material becomes a `UsdShade.Material` prim with physical properties stored as `customData`.

No manual USD editing is needed — SysML is the single source of truth.

> **Note:** Visual shaders (PBR appearance) will be added later via Omniverse. For now, materials carry only physical data.

In [ ]:
generate_material_library(str(JSON_MATERIALS), str(MAT_LIBRARY_PATH))

print(f"Generated material library: {MAT_LIBRARY_PATH}")
print()

# Show the generated USDA content
print(MAT_LIBRARY_PATH.read_text(encoding="utf-8"))

## Step 6 — Generate USD Assets

For each vetted part, we generate:
- **Geometry layer** (`*_geom.usda`) — placeholder box geometry
- **Component layer** (`<Part>.usda`) — references geometry, binds material from the shared library

Material binding uses `materialRef` to resolve against the generated `lunar_materials.usda`.

In [ ]:
builder = USDBuilder(
    vetted_parts,
    database_dir=str(ROOT / "database"),
    overwrite=True,
    use_paths_from_vetted=False,
)

outputs = builder.build_all_parts()

print("Generated USD assets:")
for name, paths in outputs.items():
    print(f"  {name}:")
    for kind, path in paths.items():
        print(f"    {kind}: {path}")

## Step 7 — Assemble Full USD Scene

We instantiate the vetted hierarchy into a single USD stage
using component references and transforms.

The result is a complete system assembly under `/World/`.

In [ ]:
scene_path = builder.write_assembly_scene(
    scene_name="HabitationAssembly.usda",
    root_name="HabitationModule",
    include_root_as_instance=True,
    instanceable=False,
    debug_refs=True,
)

print(f"Assembly scene written to: {scene_path}")

## Step 8 — Inspect the Result

Let's verify the generated USD scene by opening it and inspecting the prim hierarchy.

In [ ]:
stage = Usd.Stage.Open(scene_path)

print("USD Scene Hierarchy:")
for prim in stage.Traverse():
    depth = len(prim.GetPath().GetString().split("/")) - 1
    indent = "  " * depth
    print(f"{indent}{prim.GetName()} ({prim.GetTypeName()})")

In [ ]:
# Verify material bindings
print("Material Bindings:")
for prim in stage.Traverse():
    binding_api = UsdShade.MaterialBindingAPI(prim)
    mat, _ = binding_api.ComputeBoundMaterial()
    if mat:
        print(f"  {prim.GetPath()} -> {mat.GetPath()}")

## Summary

The pipeline successfully:

1. Parsed two SysML v2 models (system structure + material definitions)
2. Exported structured JSON with `materialRef` identifiers
3. Vetted the system graph and material references
4. Auto-generated a USD material library with physical properties from SysML
5. Built USD components with proper material bindings
6. Assembled a complete USD scene

### Architecture Principles

| Concern | Source of Truth | Tool |
|---|---|---|
| System structure & hierarchy | SysML | Parser + Exporter |
| Material identity & physical properties | SysML | Material library generator |
| Material visual appearance (shaders) | Omniverse (TODO) | Manual authoring |
| CAD geometry | STEP → USD via Omniverse (TODO) | Omniverse import |
| Integration & assembly | USD | S24 pipeline |

### View the result

```bash
usdview database/scenes/HabitationAssembly.usda
```